In [ ]:
import os
# path = r"D:\merged_v1(2)\merged_v1"
# os.chdir(path)
os.getcwd()

In [ ]:

import pandas as pd
import numpy as np
import os
import warnings
import requests
from tqdm import tqdm
warnings.filterwarnings('ignore')
from urllib.parse import urlparse
import re
import time
import urllib.parse

import datetime
import io
# from io import BytesIO
from tika import parser
import PyPDF2
from tqdm import tqdm  
import random
import string
import tldextract

import sqlite3


import scrapy.crawler as crawler
from scrapy.crawler import CrawlerProcess
from pdf_downloader.spiders.pdf_spider import PDFSpider
# from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor,defer
from scrapy.utils.project import get_project_settings
from scrapy.exceptions import CloseSpider

### Read company file and get related webpages using Google API

In [ ]:
# read company info file
df = pd.read_excel('Result_HERO_Companies w NZ Targets (06.06.23).xlsx',sheet_name = 'NZT DATA')

In [ ]:
def get_domain(url):
    """
    get company site domain
    """
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    if domain.startswith('www.'):
        domain = domain[4:]  # Remove 'www.' if present

    if "/" not in domain:
        # If ".com" is not in the url, return the original url
        return domain
    split_domain = domain.split("/", 1)

    result = split_domain[0]
    return result

In [ ]:
df_results = pd.DataFrame(columns=['result_1', 'result_2', 'result_3'])

api_endpoint = 'https://api.scaleserp.com/search'   
api_key = ''

def search_and_filter_results(terms, company_web):
    """
    Using Google API query and top3 into dateframe
    """
    url = f"{api_endpoint}?api_key={api_key}"
    # query = f"{terms} site:{company_web} filetype: pdf "
    query = f"{terms} site:{company_web}"
    params = {
        'q': query,
        # 'hl': 'en',
        'num': '3'
    }
    print(query)
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'organic_results' in data:
            results = data['organic_results']
            print([result for result in results][:3])
            domain = tldextract.extract(company_web).domain
            filtered_results = [result for result in results if tldextract.extract(result['domain']).domain in domain]
            return filtered_results[:3]
    return []


In [ ]:
for index, row in df.iterrows():
    terms = '(sustainability OR climate OR ESG) report 2022 pdf'
    company_web = row['website']
    domain = get_domain(row['website'])
    filtered_results = search_and_filter_results(terms, domain)
    print(filtered_results)
    if filtered_results:
        result_urls = [result['link'] for result in filtered_results]
        df_results.loc[index, 'id_code'] = row['id_code']
        df_results.loc[index, 'result_1'] = result_urls[0] if len(result_urls) > 0 else np.nan
        df_results.loc[index, 'result_2'] = result_urls[1] if len(result_urls) > 1 else np.nan
        df_results.loc[index, 'result_3'] = result_urls[2] if len(result_urls) > 2 else np.nan
        df_results.loc[index, 'search_query'] = terms + " " + "site:" + domain
        df_results.loc[index, 'company_mainsite'] = company_web
        df_results.loc[index, 'domain'] = domain
        df_results.loc[index, 'actor_name'] = df.loc[index, 'name']

df['domain']= df.website.apply(get_domain)
sub_df = df[['id_code','website','name','domain']]
new_column_names = ['id_code','company_mainsite','actor_name','domain']
sub_df = sub_df.rename(columns=dict(zip(sub_df.columns, new_column_names)))
df_results = pd.merge(sub_df,df_results, how='left', on = ['id_code','company_mainsite','actor_name','domain']) 

### Check whether API returned urls contains 2023 reports and store urls in a database

In [ ]:
# agents list 
user_agents = [
    # Google Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',

    # Mozilla Firefox
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0',

    # Apple Safari
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36 Edg/94.0.992.31',

    # Microsoft Edge
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.29',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.29',

    # Opera
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 OPR/63.0.3368.71',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 OPR/63.0.3368.71',

    # More Google Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',

    # More Mozilla Firefox
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:97.0) Gecko/20100101 Firefox/97.0',

    # More Apple Safari
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.3 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36 Edg/95.0.1020.30',

    # More Microsoft Edge
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 Edg/97.0.1072.62',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 Edg/97.0.1072.62',

    # More Opera
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 OPR/64.0.3417.83',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 OPR/64.0.3417.83',
]

headers = {
    'User-Agent': random.choice(user_agents)
}

additional_user_agents = [
    # Google Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_16) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36',

    # Mozilla Firefox
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_16; rv:98.0) Gecko/20100101 Firefox/98.0',

    # Apple Safari
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_16) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36 Edg/98.0.1075.0',

    # Microsoft Edge
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36 Edg/98.0.1075.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_16) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36 Edg/98.0.1075.0',

    # Opera
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36 OPR/65.0.3467.48',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_16) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4700.0 Safari/537.36 OPR/65.0.3467.48',
]

additional_headers = {
    'User-Agent': random.choice(additional_user_agents)
}

In [ ]:
def check_pdf_url(url):
    """
    Check url is PDF
    """
    if urllib.parse.urlparse(url).port == 443:
        time.sleep(2)  # adjust the sleep time as needed
    response = requests.head(url)
    content_type = response.headers.get('content-type')
    if content_type and content_type.lower() == 'application/pdf':
        return True
    return False

In [ ]:
def strip_punctuation_and_newlines(text):
    """
    remove punctuation and newlines in the text
    """
    chinese_punctuation = "，。！？【】（）％＃＠＆１２３４５６７８９０：；“”‘’《》"
    translator = str.maketrans('', '', string.punctuation + chinese_punctuation)
    stripped_text = text.translate(translator)
    stripped_text = stripped_text.replace('\n', '')
    return stripped_text

In [ ]:
def check_pdf_file_for_keyword(url, year):
    """
    read pdf url to check whether pdf is created on 2023 and contains keywords
    """

    # add a sleep time if port is 443
    if urllib.parse.urlparse(url).port == 443:
        time.sleep(random.uniform(1.0, 5.0)) # adjust the sleep time as needed
    response = requests.get(url, verify=False)
    parsed = parser.from_buffer(response.content)
    text = parsed['content']
    create_date = None
    text_short = strip_punctuation_and_newlines(text)[:300]
    try:
        pdf_content = io.BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_content)
        metadata = pdf_reader.metadata
        create_date = metadata.get('/CreationDate')
        create_date = int(create_date[2:6])
        print(create_date)
    except:
        if '2022' in text_short:
            create_date = 2023
        elif '2021' in text_short:
            create_date = 2022
        elif '2020' in text_short:
            create_date = 2021
            print(create_date)
    if len([ i for i in patterns + search_terms if i in text_short.lower()])>0 \
        and (create_date in year)\
        and ('report' in text_short.lower()):
        return True
    else:
        return False

In [ ]:
def filter_out_less2page(url):
    """
    filter out pdf less than 4 pages
    """
    # add a sleep time if port is 443
    if urllib.parse.urlparse(url).port == 443:
        time.sleep(random.uniform(3.0, 7.0)) # adjust the sleep time as needed
    response = requests.get(url, verify=False, headers=headers, timeout=60)
    try:
        pdf_content = io.BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_content)
        page_count = len(pdf_reader.pages)
    except:
        page_count = 4
    if page_count > 3:
        return True
    return False

In [ ]:
def get_pdf_create_date_page_num(pdf_url):
    """
    get pdf create date and page number
    """
    if urllib.parse.urlparse(pdf_url).port == 443:
        time.sleep(random.uniform(2.0, 7.0))  # adjust the sleep time as needed
    try:
        response = requests.get(pdf_url,verify=False, headers=additional_headers, timeout=60)
        pdf_content = io.BytesIO(response.content)
        pdf_reader = PyPDF2.PdfReader(pdf_content)
        metadata = pdf_reader.metadata
        create_date = metadata.get('/CreationDate')
        create_date = int(create_date[2:6])
        page_count = len(pdf_reader.pages)
        return create_date, int(page_count) if page_count else 0
    except:
        try: 
            if urllib.parse.urlparse(pdf_url).port == 443:
                time.sleep(random.uniform(4.0, 6.0))  # adjust the sleep time as needed
            response = requests.get(pdf_url, verify=False, headers=headers, timeout=60)
            parsed = parser.from_buffer(response.content)
            text = parsed['content']
            pdf_content = io.BytesIO(response.content)
            pdf_reader = PyPDF2.PdfReader(pdf_content)
            page_count = len(pdf_reader.pages)
            text_short = strip_punctuation_and_newlines(text)[:300]
            if '2022' in text_short:
                return 2023, int(page_count) if page_count else 0
            elif '2021' in text_short:
                return 2022, int(page_count) if page_count else 0
            elif '2020' in text_short:
                return 2021, int(page_count) if page_count else 0          
            elif '2019' in text_short:
                return 2020, int(page_count) if page_count else 0           
        except:
            return 2022,1

In [ ]:
def find_max_url(row, dictionary):
    """
    Find the latest year pdf url with the longest pages
    """
    allow_domain = row
    # allowed_domain = get_domain(row["website"])
    nested_dict = dictionary.get(allow_domain, {})
    max_first_value = float('-inf')
    max_second_value = float('-inf')
    max_url = None

    for url, (first_value, second_value) in nested_dict.items():
        if first_value > max_first_value:
            max_first_value = first_value
            max_url = url
            # print(max_first_url)

        if first_value == max_first_value and second_value > max_second_value:
            max_second_value = second_value
            max_url = url


    return max_url

In [ ]:

"pdf url keywords filter"
patterns = [
    "sustain",
    "esg",
    "environment",
    "climate",
    "green",
    "responsibility",
    "csr",
    "tcfd",
    "sgr",
    "integrated",
    "integrado",
    "annual"
]

"pdf file keywords filter"
search_terms = [ "net zero", "social", "governance report",'commitment']

current_year = datetime.datetime.now().year
numbers = [str(year)[-2:] for year in range(current_year-1, current_year+1)]
regex_patterns = [re.compile(rf"(?=.*{keyword})(?=.*{number})", re.IGNORECASE) for number in numbers for keyword in patterns]

In [ ]:
import threading

class TimeoutException(Exception):
    pass

def run_crawler(index, row):
    """
    find whether the api returned url contains keywords and is created on 2023 
    """
    keywords_url_list = []
    pdf_keywords_url_dict = {}

    for col in ["result_1", "result_2", "result_3"]:
        url = row[col]
        domain = row["domain"]
        actor_name = row["actor_name"]
        print(actor_name)
        print(domain)

        def task():
            nonlocal keywords_url_list, pdf_keywords_url_dict
            if isinstance(url, str) and ('.pdf' in url or check_pdf_url(url)):
                if url not in keywords_url_list and filter_out_less2page(url):
                    print("PDF more than 3 pages")
                    for regex_pattern in regex_patterns:
                        if regex_pattern.search(url):
                            keywords_url_list.append(url)
                            print("URL contains keywords!")
                            break
                    else:
                        print("URL doesn't contain keywords, parsing the pdf....") 
                        if check_pdf_file_for_keyword(url, [2023]) == True:
                            print("PDF contains keywords!")
                            keywords_url_list.append(url)
                    print(keywords_url_list)

                    if len(list(set(keywords_url_list))) > 1:
                        for pdf_url in keywords_url_list:
                            new_dict = {domain: {pdf_url: get_pdf_create_date_page_num(pdf_url)}}
                            for key, value in new_dict.items():
                                if key in pdf_keywords_url_dict:
                                    pdf_keywords_url_dict[key].update(value)
                                else:
                                    pdf_keywords_url_dict[key] = value
                                print(pdf_keywords_url_dict)
                                df_results.at[index, 'urls'] = find_max_url(df_results.at[index, 'domain'],pdf_keywords_url_dict)
                                df_results.at[index, 'crawled'] = 'yes'

                    else:
                        try:
                            df_results.at[index, 'urls'] = keywords_url_list[0]
                        except:
                            print("No PDF url created in 2023 and contains keywords....")
                        df_results.at[index, 'crawled'] = 'yes'
                else:
                    print("PDF less than 3 pages")
                
        thread = threading.Thread(target=task)
        thread.start()

        # Wait for 180 seconds (3 minutes) for task to complete, otherwise assign 'Timeout, retry failed'
        thread.join(60)
        if thread.is_alive():
            df_results.at[index, 'urls'] = 'Timeout, retry failed'
            print(f"Timeout, retry failed for index {index}")
            continue
    try:
        print(f"Final picked url is {df_results.at[index, 'urls']}")
    except:
        print(f"No url pick")


In [ ]:
print("crapy data from google start")

df_results['urls'] = None

crawl_deferreds = []
for index, row in df_results.iterrows():
    run_crawler(index, row)


In [ ]:
# filter Google returned pdf urls and store in the database
con = sqlite3.connect('data2.db')
df_results.to_sql(name="company",con=con,index=False, if_exists="append") # if need to append to original database
# df_results.to_sql(name="company",con=con,index=False, if_exists="replace") # if need to replace with other database
con.close()
df_results.urls.values
print("scrapying data from google pdf urls end")

### Initiate spiders to crawl webpage urls

In [ ]:
# first spider starts crawling google returned top3 urls; if not found, automatically second spider starts crawling from main site
from run_spider import run_spider

print("scrapying data from spider start")
run_spider()
print("scrapying data from spider end")

print("success!! ")

In [ ]:
con = sqlite3.connect('data2.db')
# sql = "select * from company "
print("check successful crawled urls...")
sql =  "select * from company where not (urls is null or urls =='') and (urls != 'Timeout, retry failed')"
df = pd.read_sql(sql,con=con)
df[['domain','urls','crawled']]
df.urls.values
df.shape
con.close()

### Compare spider crawl urls with API urls and take most recent one

In [ ]:
def process_row(row):
    """

    """
    print(f"{row['actor_name']} is checking....")
    if (row['urls'] is not None) and (row['urls'] in [row['result_1'], row['result_2'], row['result_3']]):
        row['final_urls'] = row['urls']
    elif row['urls'] == 'Timeout, retry failed':
        row['final_urls'] = row['urls']
    else:
        urls = [row['urls'], row['result_1'], row['result_2'], row['result_3']]
        create_dates = []
        page_counts = []
        pdf_urls = []
        for i, url in enumerate(urls):                
            if isinstance(url, str) and (url is not None) and ('.pdf' in url or check_pdf_url(url)):
                print(url)
                for regex_pattern in regex_patterns:
                    if regex_pattern.search(url):
                        print("URL contains keywords!")
                        pdf_urls.append(url)
                        create_date, page_count = get_pdf_create_date_page_num(url)
                        create_dates.append(create_date)
                        page_counts.append(page_count)
                        break
                else:
                    print("URL doesn't contain keywords, parsing the pdf....") 
                    if check_pdf_file_for_keyword(url, [2021, 2022, 2023]) == True:
                        print("PDF contains keywords!")
                        pdf_urls.append(url)
                        create_date, page_count = get_pdf_create_date_page_num(url)
                        create_dates.append(create_date)
                        page_counts.append(page_count)

        print(create_dates)
        print(page_counts)
        try:
            max_create_date = max(create_dates)
            urls_with_max_create_date = [url for url, date in zip(pdf_urls, create_dates) if date == max_create_date]
            print(urls_with_max_create_date)
            max_page_count_url = max(urls_with_max_create_date, key=lambda url: page_counts[pdf_urls.index(url)])
            print(f"final url is {max_page_count_url}")
            row['final_urls'] = max_page_count_url

        except:
            print("There is no url meet our condition and let final url blank")
    
    print(f"{row['actor_name']} finished------")

    return row

In [ ]:
print('Start checking spider urls.....')
df['final_urls'] = None

df = df.apply(process_row, axis=1)
print('Finish checking spider urls')

In [ ]:
df[df.urls != df.final_urls][['final_urls','urls']].values

### Extract txt and store txts in "txt" folder

In [ ]:
from urls_to_txt import pdf_to_text

print('Start extracting pdf to txt files.....')
df = pdf_to_text(df, 'final_urls')

# df.to_csv("company_pdf_txt.csv", index=False)
print("All pdf urls finished extraction!!!!!")

In [ ]:
df_origin = pd.read_excel('Result_HERO_Companies w NZ Targets (06.06.23)_final.xlsx',sheet_name = 'Sheet1')

df_merged = pd.merge(df_origin,df[['id_code','domain','final_urls','txt_name']], how='left', on = ['id_code'])  

df_merged.to_csv("company_pdf_txt_merged.csv", index=False)
